<a href="https://colab.research.google.com/github/wqiu96/summer_project/blob/master/src/value_iter_dirichlet_1d_v01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import time
start_time = time.time()

from tqdm import tqdm
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D 

__Goal__

Implement value iteration on 1d HJB with Dirichlet data and verify with the explicit solution on Whittle’s “Flypaper”

__Ref__

- [1] Chap V.7 of [Roger and Williams 2000] 

- [2] P97, Example 4 of [Kushner and Dupuis 2001]

- [previous version](https://github.com/songqsh/foo1/blob/master/src/value_iter_dirichlet_1d.ipynb)


__Problem setup__

We want to solve a d-dimensions HJB
$$\inf_a \{\sum_{i=1}^db_i(x,a)  \frac{\partial v(x)}{\partial x_i}  + \frac 1 2 \sum_{i,j=1}^{d} \hat \sigma_{ij} \frac{\partial^2 v(x)}{\partial x_i \partial x_j} - \lambda v(x) + \ell(x,a)\} = 0, \quad \hbox{ where } \sigma_{ij} = \sigma*\sigma^T , \sum_{i=1}^{d}x_i^2\leq1$$
with Dirichlet data
$$v(x_1,x_2,..,x_d) = g(x_1,x_2,..,x_d), \quad \hbox{ when } \sum_{i=1}^{d}x_i^2 = 1$$




__MDP discretization__

Approximation:
$$
\frac{\partial v(x)}{\partial x_i} = \frac{v(x+e_ih)-v(x-e_ih)}{2h}
$$
when $i = j$
$$
\frac{\partial^2 v(x)}{\partial x_i^2} = \frac{v(x+e_ih)-2v(x)+v(x-e_ih)}{h^2}
$$
when $i \neq j$
$$
\frac{\partial^2 v(x)}{\partial x_i^2} = \frac{v(x+e_ih+e_jh)-v(x+e_ih-e_jh)-v(x-e_ih+e_jh)+v(x-e_ih-e_jh)}{4h^2}
$$
then
$$\inf_a \{v(x)(-\lambda-\sum_{i=1}^{d}\frac{\hat \sigma_{ii}}{h^2})+\sum_{i=1}^{d}v(x+e_ih)\frac{b_i(x,a)h+\sigma_{ii}}{2h^2}+\sum_{i=1}^{d}v(x-e_ih)\frac{-b_i(x,a)h+\sigma_{ii}}{2h^2}+\sum_{i,j=1,i\neq j}^{d}(v(x+e_ih+e_jh)-v(x+e_ih-e_jh)-v(x-e_ih+e_jh)+v(x-e_ih-e_jh))\frac{\hat \sigma_{ij}}{8h^2}+ \ell(x,a)\} = 0
$$
Set
$$
c_1 = \lambda+\sum_{i=1}^{d}\frac{\hat \sigma_{ii}}{h^2}
$$
$$
\Lambda = \frac{\sum_{i=1}^{d}\frac{\lambda h+\sigma_{ii}}{h^2}}{\sum_{i=1}^{d}\frac{\sigma_{ii}}{h^2}}=\frac{\lambda h+\sum_{i=1}^{d}{\sigma_{ii}}}{\sum_{i=1}^{d}\sigma_{ii}}
$$
Then
$$
p^h(x+e_ih|x_i, a_i) = \Lambda(\frac{b_i(x,a) + \hat \sigma_{ii}}{2c_1h^2})
$$
$$
p^h(x-e_ih|x_i, a_i) = \Lambda(\frac{-b_i(x,a) + \hat \sigma_{ii}}{2c_1h^2})
$$
$$
 \hat \ell(x,a)=\sum_{i,j=1,i\neq j}^{d}(v(x+e_ih+e_jh)-v(x+e_ih-e_jh)-v(x-e_ih+e_jh)+v(x-e_ih-e_jh))\frac{\hat \sigma_{ij}}{8h^2}+ \ell(x,a)
$$
so 
$$
v(x) = \inf_a\{\sum_{i=1}^{d}p^h(x+e_ih|x_i, a_i)v(x+e_ih) +\sum_{i=1}^{d}p^h(x-e_ih|x_i, a_i)v(x-e_ih) + \hat \ell(x,a) \}
$$

__Rk__

There is essential difference of above MDP approximation using central FDM from the one
using upward FDM provided by P97 of [2].

In [0]:
#PDE to be solved
class pde:
  def __init__(self):
    #pde config
    self.name = 'HJB nd'
    self.dim = 2
    self.LIM = 1 #limit for state


    self.lambda_ = 0. #discount rate
    self.sigma = np.eye(self.dim) #diffusion coefficient matrix
 
    self.drift = lambda x,a: a  #drift coefficient function

    self.run_cost = lambda x, a: self.dim + 2*sum([i**2 for i in x]) + (1/2)*sum([j**2 for j in a])

    self.term_cost = lambda x: -sum([i**2 for i in x])
    
    #pde exact solution if available
    self.exact_sol = self.term_cost

    

In [0]:
def mdp_space_config(self, 
                     dim = 2,
                     NUM = 5 #num of meshes in one unit state space
                    ):
  self.NUM = NUM
  
  self.dim = dim
  
  self.h = 1./self.NUM #mesh size in state space
  
  self.L_LIM = 1
  
  self.x_space = np.array([i*self.h for i in range(-self.NUM*self.L_LIM, self.NUM*self.L_LIM)]) # x state space, for all dimensions, x_d have the same range,
  # range is more accurate than np.arange
  #self.x_space = np.repeat(np.expand_dims(space, axis = 0) ,self.dim , axis = 0) # x state space
  
  self.a_space = np.array([i*self.h for i in range(-2*self.NUM*self.L_LIM, 2*self.NUM*self.L_LIM)])
  
  self.del_ind_space = np.array([-1,1]) #space of delta (1-step) index: -1 means left, 1 means right move

  #q-table and state value initialization
  print(self.x_space)
  self.s_val = np.zeros(self.x_space.size*np.ones(self.dim, np.int))
  self.q_table = np.zeros(np.append(self.x_space.size*np.ones(self.dim, np.int), self.a_space.size*np.ones(self.dim, np.int)))
  
  print('>>>>> q_table size is %i' %(self.q_table.size))

  #s_val and q-table terminal setup
  for i in range(self.s_val.size):  # visit all x States
    k = self.dim - 1
    x_d = []          
    s_index = []
    Limit = 0
    while k >= 0:
      x_d.append(self.x_space[(i//(self.x_space.size**k))%self.x_space.size])
      s_index.append((i//(self.x_space.size**k))%self.x_space.size)
      Limit += self.x_space[(i//(self.x_space.size**k))%self.x_space.size]**2
      k -= 1
    if self.L_LIM - np.sqrt(self.dim*self.h**self.dim) <= Limit <= self.L_LIM + np.sqrt(self.dim*self.h**self.dim): # the ter
      self.s_val[tuple(s_index)] = self.term_cost(x_d)
      print(x_d)
      for j in range(self.a_space.size**self.dim):
        m = self.dim - 1
        a_index = []
        while m >= 0:
          a_index.append((i//(self.a_space.size**m))%self.a_space.size)
          m -= 1
        self.q_table[tuple(s_index + a_index)] = self.term_cost(x_d)
  
  
pde.mdp_space_config = mdp_space_config

In [0]:
#transition probability
#output is probability (np array) on del_ind_space 
#central fdm

def mdp_trans_prob_central(self,x,a,sigma):
  tp = []
  b_ = self.drift(x,a)
  
  c1 = self.lambda_ + sum(sigma.diagonal())/(self.h**2)
  Lambda = 1 + self.lambda_*sum(sigma.diagonal())/(self.h**2)
  
  tp.append(Lambda*((b_*self.h + sigma.diagonal())/(2*c1*self.h**2)))
  tp.append(Lambda*((-1*b_*self.h + sigma.diagonal())/(2*c1*self.h**2)))
  #tp.append(sigma/(8*c1*self.h**2))
  
  #correction on prob
#  if tp[1]<0:
#    tp = tp - tp[1]
#    tp = tp/tp.sum()
#    print('>>>>corrected probability due to negativity')

  return tp  

pde.mdp_trans_prob_central = mdp_trans_prob_central

In [0]:
def Dif_array(self,s_index):        #calculate s_val(x+e_i*h) and s_val(x-e_i*h)
  res_plus = np.zeros(self.dim)
  res_mins = np.zeros(self.dim)
  for i in range(self.dim):
    temp_plus = s_index
    temp_mins = s_index
    temp_plus[i] = s_index[i] + 1
    temp_mins[i] = s_index[i] - 1
    res_plus[i] = self.s_val[tuple(temp_plus)]
    res_mins[i] = self.s_val[tuple(temp_mins)]
  
  return res_plus,res_mins

pde.Dif_array = Dif_array

In [0]:
def Dif_martix(self,s_index):        #calculate s_val(x+e_i*h-e_j*h), s_val(x-e_j*h+e_i*h), s_val(x+e_j*h+e_i*h), s_val(x-e_j*h-e_i*h)
  res_plus_plus = np.zeros((self.dim,self.dim))
  res_mins_mins = np.zeros((self.dim,self.dim))
  res_plus_mins = np.zeros((self.dim,self.dim))
  res_mins_plus = np.zeros((self.dim,self.dim))
  for i in range(self.dim):
    for j in range(self.dim):
      if i != j:
        temp_plus_plus = s_index
        temp_mins_mins = s_index
        temp_plus_mins = s_index
        temp_mins_plus = s_index
        
        temp_plus_plus[i] = s_index[i] + 1
        temp_plus_plus[j] = s_index[j] + 1
        
        temp_mins_mins[i] = s_index[i] - 1
        temp_mins_mins[j] = s_index[j] - 1
        
        temp_plus_mins[i] = s_index[i] + 1
        temp_plus_mins[j] = s_index[j] - 1
        
        temp_mins_plus[i] = s_index[i] - 1
        temp_mins_plus[j] = s_index[j] + 1

        res_plus_plus[i][j] = self.s_val[tuple(temp_plus_plus)]
        res_mins_mins[i][j] = self.s_val[tuple(temp_mins_mins)]
        res_plus_mins[i][j] = self.s_val[tuple(temp_plus_mins)]
        res_mins_plus[i][j] = self.s_val[tuple(temp_mins_plus)]
  res = res_plus_plus + res_mins_mins - res_plus_mins - res_mins_plus
  
  return res

pde.Dif_martix = Dif_martix

In [0]:
def value_iter(self, 
               TOL= 0.0001, #tolerance for iteration
               MAX_ITER= 2000 #maximum iteration
              ):
 
  err_ = 1. #error init
  
  n_iter = 0 #iteration counter

  while err_>TOL and n_iter < MAX_ITER:
    pre_s_val = self.s_val.copy()

    for i in range(self.s_val.size):  # visit all x States
      k = self.dim - 1
      x_d = []          
      s_index = []
      Limit = 0
      while k >= 0:
        x_d.append(self.x_space[(i//(self.x_space.size**k))%self.x_space.size])
        s_index.append((i//(self.x_space.size**k))%self.x_space.size)
        Limit += self.x_space[(i//(self.x_space.size**k))%self.x_space.size]**2
        k -= 1
      if Limit < self.L_LIM:
        for j in range(self.a_space.size**self.dim):
          m = self.dim - 1
          a_index = []
          a_list = []
          while m >= 0:
            a_index.append((i//(self.a_space.size**m))%self.a_space.size)
            a_list.append(self.a_space[(i//(self.a_space.size**m))%self.a_space.size])
            m -= 1
          a_ = np.array(a_list)
          tp_ = self.mdp_trans_prob_central(x_d, a_, self.sigma)
          
          c1 = self.lambda_ + sum(self.sigma.diagonal())/(self.h**2)
          res_plus,res_mins = self.Dif_array(s_index)
          res = self.Dif_martix(s_index)
          
          Pd = np.sum(np.multiply(res,self.sigma))
          run_cost_ = (self.run_cost(x_d,a_) + Pd/(8*self.h**2))/c1
          Lambda = 1 + self.lambda_*sum(self.sigma.diagonal())/(self.h**2)

          self.q_table[tuple(s_index+a_index)] = (run_cost_ + np.sum(np.multiply(tp_[0],res_plus)) + np.sum(np.multiply(tp_[1],res_mins)))/Lambda

        self.s_val[tuple(s_index)] = np.min(self.q_table[tuple(s_index)]) #sync q_table with s_val
    err_ = np.max(np.abs(pre_s_val - self.s_val))

    n_iter += 1
    if np.mod(n_iter,10) == 0:
      print('iterated ' + str(n_iter)+';')
      
  return [err_, n_iter]
      
pde.value_iter = value_iter      

In [16]:
pde1 = pde()
pde1.mdp_space_config(dim=2,NUM=10) #if NUM is chosen to be 20 or bigger number, the result is bad
err, n_iter = pde1.value_iter(TOL = 0.001, MAX_ITER = 1000)

[-1.  -0.9 -0.8 -0.7 -0.6 -0.5 -0.4 -0.3 -0.2 -0.1  0.   0.1  0.2  0.3
  0.4  0.5  0.6  0.7  0.8  0.9]
>>>>> q_table size is 640000
[-1.0, -0.30000000000000004]
[-1.0, -0.2]
[-1.0, -0.1]
[-1.0, 0.0]
[-1.0, 0.1]
[-1.0, 0.2]
[-1.0, 0.30000000000000004]
[-0.9, -0.5]
[-0.9, -0.4]
[-0.9, -0.30000000000000004]
[-0.9, 0.30000000000000004]
[-0.9, 0.4]
[-0.9, 0.5]
[-0.8, -0.7000000000000001]
[-0.8, -0.6000000000000001]
[-0.8, -0.5]
[-0.8, 0.5]
[-0.8, 0.6000000000000001]
[-0.8, 0.7000000000000001]
[-0.7000000000000001, -0.8]
[-0.7000000000000001, -0.7000000000000001]
[-0.7000000000000001, 0.7000000000000001]
[-0.7000000000000001, 0.8]
[-0.6000000000000001, -0.8]
[-0.6000000000000001, 0.8]
[-0.5, -0.9]
[-0.5, -0.8]
[-0.5, 0.8]
[-0.5, 0.9]
[-0.4, -0.9]
[-0.4, 0.9]
[-0.30000000000000004, -1.0]
[-0.30000000000000004, -0.9]
[-0.30000000000000004, 0.9]
[-0.2, -1.0]
[-0.1, -1.0]
[0.0, -1.0]
[0.1, -1.0]
[0.2, -1.0]
[0.30000000000000004, -1.0]
[0.30000000000000004, -0.9]
[0.30000000000000004, 0.9]
[0.4, 

KeyboardInterrupt: ignored

In [0]:
end_time = time.time()
print('elapsed time is %f seconds' %(end_time - start_time))

In [0]:
figure = plt.figure()
axes = Axes3D(figure)
X1 = pde1.x_space
X2 = pde1.x_space
X1, X2 = np.meshgrid(X1, X2)
Z = -(X1)**2 - (X2)**2
axes.plot_surface(X1, X2, Z,cmap='rainbow')